In [108]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import vincenty

foursquare_data = pd.read_csv("./data/foursquare-nyc-and-tokyo-check-ins/dataset_TSMC2014_NYC.csv")
climate_data = pd.read_csv("./data/1087533.csv")

/Users/frankyliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8,10,11,14,15,20,23,25,40,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [109]:
pd.options.display.max_columns = 999
climate_data.HOURLYPrecip = climate_data.HOURLYPrecip.fillna(0.00).replace('T', '0.00').str.replace(r"[a-zA-Z]",'').astype(float)

In [110]:
def closest_station(point, points):
    closest_station = ""
    closest_distance = 50000                                                 
    for station in points:
        cur_dist = vincenty(point, station).kilometers
        if cur_dist < closest_distance:
            closest_station = station_lat_long[station_lat_long['lat_long'] == station]['STATION_NAME'].values[0]
    return closest_station

station_lat_long = climate_data[['STATION_NAME', 'LATITUDE', 'LONGITUDE']].drop_duplicates()
foursquare_data['lat_long'] = [(x, y) for x,y in zip(foursquare_data['latitude'], foursquare_data['longitude'])]                                       
station_lat_long['lat_long'] = [(x, y) for x,y in zip(station_lat_long['LATITUDE'], station_lat_long['LONGITUDE'])]

foursquare_data['closestStation'] = [closest_station(pt, station_lat_long['lat_long']) for pt in foursquare_data['lat_long']]



In [111]:
climate_data.DATE = pd.to_datetime(climate_data.DATE, utc=True)
climate_data.sort_values('DATE', inplace=True)
foursquare_data.utcTimestamp = pd.to_datetime(foursquare_data.utcTimestamp, utc=True)
foursquare_data.sort_values('utcTimestamp', inplace=True)

In [112]:
climate_data['roundedDate'] = climate_data.DATE.dt.round('60min')
foursquare_data['roundedDate'] = foursquare_data.utcTimestamp.dt.round('60min')
foursquare_climate_df = pd.merge(foursquare_data, climate_data,  how='inner', left_on=['closestStation','roundedDate'], right_on = ['STATION_NAME','roundedDate'])

In [113]:
foursquare_climate_df[foursquare_climate_df.venueId == '49c54c2cf964a520ee561fe3']
foursquare_climate_df[foursquare_climate_df.venueId == '3fd66200f964a52021e41ee3']

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp,lat_long,closestStation,roundedDate,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,DAILYMaximumDryBulbTemp,DAILYMinimumDryBulbTemp,DAILYAverageDryBulbTemp,DAILYDeptFromNormalAverageTemp,DAILYAverageRelativeHumidity,DAILYAverageDewPointTemp,DAILYAverageWetBulbTemp,DAILYHeatingDegreeDays,DAILYCoolingDegreeDays,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth,DAILYAverageStationPressure,DAILYAverageSeaLevelPressure,DAILYAverageWindSpeed,DAILYPeakWindSpeed,PeakWindDirection,DAILYSustainedWindSpeed,DAILYSustainedWindDirection,MonthlyMaximumTemp,MonthlyMinimumTemp,MonthlyMeanTemp,MonthlyAverageRH,MonthlyDewpointTemp,MonthlyWetBulbTemp,MonthlyAvgHeatingDegreeDays,MonthlyAvgCoolingDegreeDays,MonthlyStationPressure,MonthlySeaLevelPressure,MonthlyAverageWindSpeed,MonthlyTotalSnowfall,MonthlyDeptFromNormalMaximumTemp,MonthlyDeptFromNormalMinimumTemp,MonthlyDeptFromNormalAverageTemp,MonthlyDeptFromNormalPrecip,MonthlyTotalLiquidPrecip,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT32Temp,MonthlyDaysWithGT32Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT1Snow,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureDate,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
47250,375,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-28 21:31:49,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-28 22:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-28 21:51:00,FM-15,OVC:08 120,10.00,NaN,53,11.7,41.0,5.3,25,-3.9,34.0,11.0,280,NaN,30.12,1.0,NaN,30.14,0.00,30.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
47251,375,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-28 21:31:49,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-28 22:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-28 22:00:00,FM-12,NaN,9.94,NaN,53,11.7,41.0,5.3,25,-3.9,34.0,11.0,280,NaN,30.11,1.0,-0.03,30.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
48369,122,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-29 03:07:26,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-29 03:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-29 02:51:00,FM-15,NaN,10.00,NaN,47,8.3,40.0,4.2,29,-1.7,50.0,3.0,330,NaN,30.10,NaN,NaN,30.12,0.00,30.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [114]:
foursquare_climate_df.to_csv("foursquare_climate.csv", sep='\t')